In [1]:
from google.colab import files

files.upload()

! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d vladislavkovalev/itadatahack
!kaggle datasets download -d vladislavkovalev/itadatahack-test

Saving kaggle.json to kaggle (2).json
mkdir: cannot create directory ‘/root/.kaggle’: File exists
itadatahack.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
! mkdir data
! unzip itadatahack.zip -d data
! unzip itadatahack-test.zip -d data

In [ ]:
!pip install -q transformers==4.17.0
!pip install -q datasets==2.3.2

import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset
import re
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
def numbers_to_classes(l):
  zero_cl = [0.0] * 20
  zero_cl[int(l)-1] = 1.0
  return np.array(zero_cl, dtype=np.float)


def preprocess_data(examples):
  # take a batch of texts
  text = examples["text"]
  text = [re.sub("^[A-Za-z0-9]*$", "", text_iter.lower())
      .replace(' sep ', 'SEP') for text_iter in text]
  # encode them
  encoding = tokenizer(text, padding="max_length",
                       truncation=True, max_length=500)

  # add labels
  labels_matrix = []
  for r in examples["old_labels"]:
    labels_matrix.append(np.array(numbers_to_classes(r), dtype=float))
  # print(labels_matrix)
  encoding["labels"] = np.array(labels_matrix, dtype=float)

  return encoding


def multi_label_metrics(predictions, labels):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))

    y_pred = np.zeros(probs.shape)
    num_rows = probs.size(0)
    y_pred[range(num_rows), np.argmax(probs, axis=1)] = 1

    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics


def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result


def predict_label(text):
    encoding = tokenizer(text, padding="max_length",
                          truncation=True, max_length=300, return_tensors="pt")
    encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

    outputs = model(**encoding)
    logits = outputs.logits
    # apply sigmoid + threshold
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits.squeeze().cpu())
    predictions = np.zeros(probs.shape)
    num_rows = probs.size(0)

    return probs.argmax() + 1

In [4]:
id2label = {0: '1',
 1: '2',
 2: '3',
 3: '4',
 4: '5',
 5: '6',
 6: '7',
 7: '8',
 8: '9',
 9: '10',
 10: '11',
 11: '12',
 12: '13',
 13: '14',
 14: '15',
 15: '16',
 16: '17',
 17: '18',
 18: '19',
 19: '20'}

label2id = {'1': 0,
 '2': 1,
 '3': 2,
 '4': 3,
 '5': 4,
 '6': 5,
 '7': 6,
 '8': 7,
 '9': 8,
 '10': 9,
 '11': 10,
 '12': 11,
 '13': 12,
 '14': 13,
 '15': 14,
 '16': 15,
 '17': 16,
 '18': 17,
 '19': 18,
 '20': 19}

In [8]:
# data reader

data_train = pd.read_csv('/content/data/train_set.csv')
data_train = data_train.drop_duplicates()

In [ ]:
# lemmatization module

# stop words dictionary from nltk module
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# spaCy module lemmatization
lemmatizer = WordNetLemmatizer()

# preprocess words in sentences
def preprocess_text(text):
    tokens = text.lower().split()
    filtered_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    return ' '.join(filtered_tokens)

In [ ]:
data_train['text_column'] = data_train['text_column'].apply(preprocess_text)
data_train['Directory code'] \
  = data_train['Directory code'].astype(str)

In [10]:
# citations (references) processing 
data_train['Citations'] \
    = data_train['Citations'].apply(lambda x:
          ', '.join(x.replace('[', '')\
                    .replace(']', '')\
                    .replace("'", '')
                    .split(',')))

In [12]:
data_train['FULL_TEXT']=data_train['Citations'] + " " \
                             + data_train['Text']
data_train = data_train[['CELEX_ID', 'FULL_TEXT', 'Directory code']]

In [13]:
data_train.head(3)

,CELEX_ID,FULL_TEXT,Directory code
0,32016D1888,"OJL149, 20.5.2014, p.67, OJL131, 1.6.2000, p....",19
1,32013D0005,"OJL322, 14.12.1999, p.32, OJL164, 25.6.2008, ...",15
2,32019D1268,"OJL65, 11.3.2011, p.1 29.7.2019 EN Official Jo...",20


In [14]:
data_train.rename({'Directory code': 'OLD_LABEL'}, axis=1, inplace=True)
data_train['FULL_TEXT'] = data_train.FULL_TEXT.astype('str')

<ipython-input-14-ec3bf2c2b68a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train.rename({'Directory code': 'OLD_LABEL'}, axis=1, inplace=True)
<ipython-input-14-ec3bf2c2b68a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train['FULL_TEXT'] = data_train.FULL_TEXT.astype('str')


In [16]:
data_train.rename(
    {"CELEX_ID": "celex_id", "FULL_TEXT": "text", "OLD_LABEL": 'old_labels'},
    axis=1,
    inplace=1)

<ipython-input-16-31cc41562caa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train.rename(


In [17]:
data_train = data_train[['text', 'old_labels']]
data_train.reset_index(inplace=True)

In [19]:
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")



In [20]:
hg_dataset = Dataset(pa.Table.from_pandas(data_train))
encoded_dataset = hg_dataset.map(preprocess_data, batched=True)
encoded_dataset = encoded_dataset.train_test_split(test_size=0.3)
del(encoded_dataset['old_labels'])

  0%|          | 0/12 [00:00<?, ?ba/s]

<ipython-input-15-3c537cb38aca>:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.array(zero_cl, dtype=np.float)


In [21]:
encoded_dataset.set_format("torch")

In [22]:
model = AutoModelForSequenceClassification.from_pretrained("nlpaueb/legal-bert-base-uncased",
                                                           problem_type="multi_label_classification",
                                                           num_labels=len(data_train.old_labels.unique()),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of the model checkpoint at nlpaueb/legal-bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

In [23]:
batch_size = 16
metric_name = "f1"

In [24]:
args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name
)

In [26]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [27]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: index, text, old_labels. If index, text, old_labels are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7569
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4740


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.074337,0.851167,0.921667,0.851167
2,0.152800,0.049293,0.884956,0.939450,0.884956
3,0.059000,0.038698,0.899973,0.947354,0.899973
4,0.039600,0.034493,0.902923,0.948907,0.902923
5,0.030500,0.033742,0.905337,0.950177,0.905337
6,0.023000,0.030805,0.919281,0.957516,0.919281
7,0.019500,0.030969,0.919013,0.957375,0.919013
8,0.015700,0.030571,0.922767,0.959351,0.922767
9,0.013900,0.030300,0.919818,0.957799,0.919818
10,0.012100,0.030135,0.921158,0.958504,0.921158


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: index, text, old_labels. If index, text, old_labels are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3729
  Batch size = 16
Saving model checkpoint to bert-finetuned-sem_eval-english/checkpoint-474
Configuration saved in bert-finetuned-sem_eval-english/checkpoint-474/config.json
Model weights saved in bert-finetuned-sem_eval-english/checkpoint-474/pytorch_model.bin
tokenizer config file saved in bert-finetuned-sem_eval-english/checkpoint-474/tokenizer_config.json
Special tokens file saved in bert-finetuned-sem_eval-english/checkpoint-474/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: index, text, old_labels. If index, text, 

TrainOutput(global_step=4740, training_loss=0.039191682881946806, metrics={'train_runtime': 8013.6197, 'train_samples_per_second': 9.445, 'train_steps_per_second': 0.591, 'total_flos': 1.945126398204e+16, 'train_loss': 0.039191682881946806, 'epoch': 10.0})

In [28]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: index, text, old_labels. If index, text, old_labels are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3729
  Batch size = 16


{'eval_loss': 0.03057071939110756,
 'eval_f1': 0.9227674979887369,
 'eval_roc_auc': 0.9593513147309142,
 'eval_accuracy': 0.9227674979887369,
 'eval_runtime': 115.7481,
 'eval_samples_per_second': 32.217,
 'eval_steps_per_second': 2.022,
 'epoch': 10.0}

In [ ]:
data_test = pd.read_csv('/content/data/new_test.csv')
del(data_test['Unnamed: 0'])
del(data_test['CELEX_ID'])
data_test['Citations'] \
    = data_test['Citations'].apply(lambda x: ', '.join(x.replace('[', '').replace(']', '').split(',')))
data_test['Text'] = data_test['Citations'].str.replace("'", '') + " [SEP] " + data_test['Text']
data_test.rename({"CELEX_ID": "celex_id", 'Text': 'text'}, axis=1, inplace=True)
data_test = data_test[['text']]
data_test = data_test.reset_index(drop=True)

In [40]:
text = data_test["text"]
text = [re.sub("^[A-Za-z0-9]*$", "", text_iter.lower())
    .replace(' sep ', 'SEP') for text_iter in text]
hg_dataset = Dataset(pa.Table.from_pandas(data_test))

In [41]:
labels = [int(predict_label(text=text['text'])) for text in hg_dataset]
labels = pd.DataFrame(labels)
labels.to_csv('test_new_answ.csv')  # result csv file

In [ ]:
labels = pd.DataFrame(labels)


In [50]:
labels[0].value_counts()

1     707
18    189
11    167
3      81
15     26
13     18
16     11
5       7
10      6
7       4
19      4
6       4
4       3
17      2
12      1
9       1
Name: 0, dtype: int64